# Machine Learning Applications for Accelerators

## Exercise 1a
Build your own Bayesian Optimisation algo. 
Use the GP and data from Exercise 1 and optimise it between $[0,2π]$  



In [75]:
from math import sin, pi, ceil, floor
import matplotlib.pyplot as plt
import numpy as np

# Generate 5 test points for ground truth and plot 
x_train = np.linspace(0, 2*pi, 5)
#x_train = [0,1,2,3,4]
y_train = [sin(x) for x in x_train]

# plt.scatter(x_train, y_train, label='Ground truth')
# plt.legend()
# plt.show()

In [76]:

def rbf_kernel(x1, x2, length_scale=1.0):
    sqdist = np.subtract.outer(x1, x2) ** 2
    return np.exp(-0.5 * sqdist / length_scale**2)

K_train_train = rbf_kernel(x_train, x_train)


minn, maxx = min(x_train), max(x_train)
minn, maxx = floor(minn), ceil(maxx)


test_amount = 20*(maxx-minn)
x_test = np.linspace(minn, maxx, test_amount)

y_true_vals = [sin(el) for el in x_test]

K_test_train = rbf_kernel(x_test, x_train)
K_train_test = np.transpose( K_test_train)

K_test_test = rbf_kernel(x_test, x_test)

def mean(K_test_train, K_train_train, y_train):
    return K_test_train @ np.linalg.inv(K_train_train) @ y_train

def variance(K_test_test, K_test_train, K_train_train, K_train_test):
    return np.subtract(K_test_test, (K_test_train @ np.linalg.inv(K_train_train) @ K_train_test))


In [ ]:
from math import pi, e 

def gaussian(x, mu, sigma):
    denom = (2*pi*sigma*sigma)**0.5
    exp = -((x-mu)**2/(2*sigma*sigma))
    return e**exp/denom

meann = mean(K_test_train, K_train_train, y_train)
variancee_mat = variance(K_test_test, K_test_train, K_train_train, K_train_test)

meann = np.array(meann)
variancee_mat = np.array(variancee_mat)

variancee = np.diag(variancee_mat)

# print(meann.shape, "\n", variancee.shape)


y_test = gaussian(x_test, meann, variancee)

# plt.plot(x_test, y_test, color = "blue")
# plt.scatter(x_train, y_train, color="orange")
# plt.show()

In [ ]:
def plot_gp_with_confidence(x_train, y_train, x_test, mean_pred, var_pred, show=True, confidence=1.96):
    std_pred = np.sqrt(var_pred)  # Standard deviation is the square root of the variance

    # Plot the mean prediction
    plt.plot(x_test, y_true_vals, color='orange', label='Ground truth')
    plt.plot(x_test, mean_pred, color="blue", label="GP Mean")

    # Plot the confidence interval
    plt.fill_between(x_test, 
                     mean_pred - confidence * std_pred, 
                     mean_pred + confidence * std_pred, 
                     color="lightblue", alpha=0.5, label="95% Confidence Interval")
    
    # Plot the original data points
    plt.scatter(x_train, y_train, color="orange", label="Training Data")
    
    # Add labels and legend
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.title("Gaussian Process Regression with Confidence Intervals")
    plt.legend()
    if show:
        plt.show()

plot_gp_with_confidence(x_train, y_train, x_test, meann, variancee)

 * Define the Acquisition Function UCB method for a single data point 
    * e.g. `def acq_ucb(x):`...  
Iteratively find the optimum of $f(x)$ by optimising the Acquisition Function at each iteration.  

In [ ]:
lambdaa = 2.0
# lambdaa = 0.5
# lambdaa = 5.0
def upper_confidence_bound():
    return np.add(meann, np.multiply(lambdaa, variancee))

plot_gp_with_confidence(x_train, y_train, x_test, meann, variancee, show=False)
plt.plot(x_test, upper_confidence_bound(), color="green", label=f"UCB lambda={lambdaa}")
plt.show()